In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import math
from sklearn.model_selection import KFold
import xgboost as xgb

The following code segments are for running the historical analysis shown in figure 2 of the paper. The code does the following:

* Set input path to the data and an output path to where you want to save output data  
* Open data. The data used to run the model can be downloaded from: https://doi.org/10.5518/1152

* Set dependent variable name
* Set simulation information
* Set years you would like to analyse 
* Set XGBoost model 
* Loop over simulations and years and fit model to data in each simulation

Set inpath and outpath

In [2]:
# Path to the data.
inpath = '.../'

# Outpath where you would like to save output data.
outpath = '.../XGout/' 

Open data

In [3]:
df = pd.read_pickle(inpath + 'trainvalid.pkl')
df = pd.DataFrame(df)

In [4]:
df

,FID,geometry,centre,area_km,states,year,month,fire_number,defor,t_month,p_6_month,p_month,pasture_frac,crop_frac,savanna_frac,lai_12_month_mean
0,6938,"POLYGON ((-73.50000 -6.75000, -73.25000 -6.750...",POINT (-73.37500 -6.87500),763.931320,Amazonas,2003,1,0.0,0.000000,28.2428,713.030685,268.399323,0.000,0.000,0.000,5.258945
1,6939,"POLYGON ((-73.50000 -7.00000, -73.25000 -7.000...",POINT (-73.37500 -7.12500),763.532945,Amazonas,2003,1,0.0,1.357946,28.1812,756.387218,270.729797,0.001,0.000,0.000,5.231799
2,6940,"POLYGON ((-73.50000 -7.25000, -73.25000 -7.250...",POINT (-73.37500 -7.37500),763.120400,Acre,2003,1,0.0,8.050515,28.3756,779.980492,248.937378,0.017,0.000,0.000,5.270495
3,6941,"POLYGON ((-73.50000 -7.50000, -73.25000 -7.500...",POINT (-73.37500 -7.62500),762.693690,Acre,2003,1,0.0,4.219792,27.8924,805.984306,234.089584,0.005,0.000,0.000,5.223077
4,6942,"POLYGON ((-73.50000 -7.75000, -73.25000 -7.750...",POINT (-73.37500 -7.87500),762.252820,Acre,2003,1,0.0,3.456357,27.2764,810.648941,218.884186,0.003,0.000,0.000,5.252493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5126,29463,"POLYGON ((-44.75000 -3.00000, -44.50000 -3.000...",POINT (-44.62500 -3.12500),768.202987,Maranhao,2020,12,0.0,0.000000,30.4916,333.955908,135.318237,0.050,0.006,0.487,1.794112
5127,29657,"POLYGON ((-44.50000 -2.50000, -44.25000 -2.500...",POINT (-44.37500 -2.62500),768.530736,Maranhao,2020,12,0.0,0.924999,29.5052,351.155690,92.066948,0.043,0.001,0.008,2.633466
5128,29658,"POLYGON ((-44.50000 -2.75000, -44.25000 -2.750...",POINT (-44.37500 -2.87500),768.373982,Maranhao,2020,12,0.0,0.232154,28.4492,366.457522,118.020523,0.053,0.001,0.085,2.791391
5129,29659,"POLYGON ((-44.50000 -3.00000, -44.25000 -3.000...",POINT (-44.37500 -3.12500),768.202987,Maranhao,2020,12,0.0,1.950259,30.0412,328.087385,146.691986,0.255,0.002,0.200,2.445894


Print features/variable names. Refer to data README file for a description of variables.

In [5]:
df.columns

Index(['FID', 'geometry', 'centre', 'area_km', 'states', 'year', 'month',
       'fire_number', 'defor', 't_month', 'p_6_month', 'p_month',
       'pasture_frac', 'crop_frac', 'savanna_frac', 'lai_12_month_mean'],
      dtype='object')

Set dependent variable name

In [6]:
dep_var = 'fire_number'

Resacale featues that we will use in the simultaions.

In [7]:
df['defor'] = df['defor'] / df['area_km'] 
df['defor'] = (df['defor'] - df['defor'].min())/(df['defor'].max() - df['defor'].min())
df['p_month'] = (df['p_month'] - df['p_month'].min())/(df['p_month'].max() - df['p_month'].min())
df['t_month'] = (df['t_month'] - df['t_month'].min())/(df['t_month'].max() - df['t_month'].min())
df['p_6_month'] = (df['p_6_month'] - df['p_6_month'].min())/(df['p_6_month'].max() - df['p_6_month'].min())
df['lai_12_month_mean'] = (df['lai_12_month_mean'] - df['lai_12_month_mean'].min())/(df['lai_12_month_mean'].max() - df['lai_12_month_mean'].min())

Set simulation information: 

* 'to_drop': features to drop 
* 'sim_label': label used to save predictions for each simulation. 

Refer to paper for details about each simulation. 

In [8]:
sim_label = [
    'Sim_Clim+Def',
    'Sim_Clim',
    'Sim_Clim+LU',
    'Sim_Clim+LU+Def'
]

sim_features = [
    
    ['fire_number', 'defor', 't_month', 'p_6_month', 'p_month','lai_12_month_mean', 'year'], # 'Sim_Clim+Def'
    
    ['fire_number', 't_month', 'p_6_month', 'p_month','lai_12_month_mean', 'year'],          # 'Sim_Clim'
    
    ['fire_number', 't_month', 'p_6_month', 'p_month','lai_12_month_mean',                   # 'Sim_Clim+LU'
     'pasture_frac', 'crop_frac', 'savanna_frac', 'year'], 
    
    ['fire_number', 'defor', 't_month', 'p_6_month', 'p_month','lai_12_month_mean',          # 'Sim_Clim+LU+Def'
     'pasture_frac', 'crop_frac', 'savanna_frac', 'year']
    
]


Set years you would like to analyse. Years allowed 2003 to 2020.

In [9]:
years = np.arange(2003,2021,1)

Set Random Forest model and RMSE function

In [10]:
def xg(X, y,
       objective = 'reg:squarederror',
       n_estimators = 80,
       max_depth = 8,
       learning_rate = 0.1,
       colsample_bytree = 0.8,
       reg_alpha = 1.2,
       reg_lambda = 1.2,
       subsample = 0.9,**kwargs): 
    return xgb.XGBRegressor(objective = objective,
                            n_estimators = n_estimators,
                            max_depth = max_depth,
                            learning_rate = learning_rate,
                            colsample_bytree = colsample_bytree,
                            reg_alpha = reg_alpha,
                            reg_lambda = reg_lambda,
                            subsample = subsample).fit(X, y)

def r_mse(pred,y): 
    return round(math.sqrt(((pred-y)**2).mean()), 6)

Loop over simulations and years and fit model to data in each simulation

In [11]:
# Loop through each simulation
for sim_index, sim in enumerate(sim_label):
    # Print simulation
    print(sim)
    
    # Get featues/variables for each simulation
    sim_data = df[sim_features[sim_index]]
       
    # Loop through each year
    for yr in years:
        
        # Print year
        print(yr)
        
        # Copy simulation data
        train = sim_data.copy()
        
        # Get test data. Test set is data for year of interest.
        test = train[(train['year'] == yr)]
        
        # Use this to add test predictions 
        test_original = test.copy()
        
        # Remove test data from train data
        train = train.loc[~((train['year'] == yr)),:]
        
        # Remove 'year' from datasets as it is not needed now
        train = train.drop('year', axis=1)
        test = test.drop('year', axis=1)
        
        # X,y test dataset
        X_test, y_test = test.drop(dep_var,axis=1), test[dep_var]
        
        # Set number of k-folds 
        n_folds = 5 
    
        # Empty array to store test predictions for each k-fold   
        preds_test_ar = np.ones((n_folds,len(X_test)))
        
        # Define splits into k-folds
        kf = KFold(n_splits=n_folds, random_state=42, shuffle=True) 
        kf.get_n_splits(train) # returns the number of splitting iterations in the cross-validator
    
        index_count  = 0
        for train_index, valid_index in kf.split(train):
            index_count += 1
            print(index_count-1)
        
            # Get train and valid set for fold
            X_train, X_valid = train.iloc[train_index], train.iloc[valid_index]
            y_train, y_valid = X_train[dep_var], X_valid[dep_var]  
            
            # Drop dependent/target variable 
            X_train, X_valid = X_train.drop(dep_var,axis=1), X_valid.drop(dep_var,axis=1) 
            
            # Fit model   
            m = xg(X_train,y_train)
            
            # Get fire count predictions for train, valid and test sets
            preds_train = m.predict(X_train)
            preds_valid = m.predict(X_valid)
            preds_test =  m.predict(X_test)

            # Print RMSE for train, valid and test sets
            print('Train RMSE in = ' + str(r_mse(preds_train, y_train)))
            print('Valid RMSE in = ' + str(r_mse(preds_valid, y_valid)))
            print('Test RMSE in ' + str(yr) + '= ' + str(r_mse(preds_test, y_test))) 

            # Store test fire count predictions    
            preds_test_ar[index_count-1,:] = preds_test
        
            # Average and save fire count predictions
            if index_count-1 == 4:  

                # Average test predictions across folds and save 
                test_original['mean_preds'] = np.round(preds_test_ar.mean(axis=0))
                test_original.to_csv(outpath + 'test_preds_'+sim+'_'+str(n_folds)+'folds_'+str(yr)+'.csv', index=False)
    

    

Sim_Clim+Def
2003
0
Train RMSE in = 5.848876
Valid RMSE in = 6.953893
Test RMSE in 2003= 8.318193
1
Train RMSE in = 5.858103
Valid RMSE in = 7.118663
Test RMSE in 2003= 8.261812
2
Train RMSE in = 5.826201
Valid RMSE in = 7.417874
Test RMSE in 2003= 8.339205
3
Train RMSE in = 5.8695
Valid RMSE in = 7.157003
Test RMSE in 2003= 8.282473
4
Train RMSE in = 5.943584
Valid RMSE in = 6.691726
Test RMSE in 2003= 8.326632
2004
0
Train RMSE in = 5.772994
Valid RMSE in = 6.848171
Test RMSE in 2004= 9.76456
1
Train RMSE in = 5.780803
Valid RMSE in = 6.962302
Test RMSE in 2004= 9.75958
2
Train RMSE in = 5.758429
Valid RMSE in = 7.296216
Test RMSE in 2004= 9.814693
3
Train RMSE in = 5.774158
Valid RMSE in = 7.092084
Test RMSE in 2004= 9.668093
4
Train RMSE in = 5.811943
Valid RMSE in = 6.749953
Test RMSE in 2004= 9.821328
2005
0
Train RMSE in = 5.671123
Valid RMSE in = 6.726133
Test RMSE in 2005= 11.450867
1
Train RMSE in = 5.668852
Valid RMSE in = 6.850099
Test RMSE in 2005= 11.458358
2
Train RMSE i

Train RMSE in = 7.235874
Valid RMSE in = 8.542793
Test RMSE in 2005= 14.972069
3
Train RMSE in = 7.300221
Valid RMSE in = 8.194563
Test RMSE in 2005= 14.931335
4
Train RMSE in = 7.372075
Valid RMSE in = 7.754598
Test RMSE in 2005= 14.924072
2006
0
Train RMSE in = 7.799874
Valid RMSE in = 8.411769
Test RMSE in 2006= 9.351006
1
Train RMSE in = 7.780497
Valid RMSE in = 8.582175
Test RMSE in 2006= 9.335994
2
Train RMSE in = 7.702227
Valid RMSE in = 8.867372
Test RMSE in 2006= 9.362985
3
Train RMSE in = 7.745573
Valid RMSE in = 8.69107
Test RMSE in 2006= 9.320088
4
Train RMSE in = 7.807772
Valid RMSE in = 8.16372
Test RMSE in 2006= 9.357823
2007
0
Train RMSE in = 7.468057
Valid RMSE in = 8.172348
Test RMSE in 2007= 13.769175
1
Train RMSE in = 7.492281
Valid RMSE in = 8.277723
Test RMSE in 2007= 13.761447
2
Train RMSE in = 7.467524
Valid RMSE in = 8.360759
Test RMSE in 2007= 13.771449
3
Train RMSE in = 7.506336
Valid RMSE in = 8.328562
Test RMSE in 2007= 13.765693
4
Train RMSE in = 7.508759


Train RMSE in = 6.840831
Valid RMSE in = 7.389335
Test RMSE in 2007= 12.66935
2008
0
Train RMSE in = 7.133193
Valid RMSE in = 8.046138
Test RMSE in 2008= 6.253047
1
Train RMSE in = 7.132366
Valid RMSE in = 8.258055
Test RMSE in 2008= 6.168273
2
Train RMSE in = 7.149823
Valid RMSE in = 8.096376
Test RMSE in 2008= 6.159584
3
Train RMSE in = 7.113583
Valid RMSE in = 8.12848
Test RMSE in 2008= 6.165801
4
Train RMSE in = 7.146932
Valid RMSE in = 7.805775
Test RMSE in 2008= 6.158322
2009
0
Train RMSE in = 7.217844
Valid RMSE in = 8.101113
Test RMSE in 2009= 4.61523
1
Train RMSE in = 7.179877
Valid RMSE in = 8.278141
Test RMSE in 2009= 4.64862
2
Train RMSE in = 7.168793
Valid RMSE in = 8.16067
Test RMSE in 2009= 4.584654
3
Train RMSE in = 7.178417
Valid RMSE in = 8.191642
Test RMSE in 2009= 4.612776
4
Train RMSE in = 7.233623
Valid RMSE in = 7.838278
Test RMSE in 2009= 4.602197
2010
0
Train RMSE in = 6.903929
Valid RMSE in = 7.744757
Test RMSE in 2010= 11.741655
1
Train RMSE in = 6.924628
Val

Train RMSE in = 5.398679
Valid RMSE in = 6.585931
Test RMSE in 2010= 11.615983
1
Train RMSE in = 5.425599
Valid RMSE in = 6.424888
Test RMSE in 2010= 11.697064
2
Train RMSE in = 5.397902
Valid RMSE in = 6.449809
Test RMSE in 2010= 11.718555
3
Train RMSE in = 5.402517
Valid RMSE in = 6.580501
Test RMSE in 2010= 11.716977
4
Train RMSE in = 5.41238
Valid RMSE in = 6.493572
Test RMSE in 2010= 11.608392
2011
0
Train RMSE in = 5.726907
Valid RMSE in = 6.972062
Test RMSE in 2011= 3.56574
1
Train RMSE in = 5.753443
Valid RMSE in = 7.018883
Test RMSE in 2011= 3.618406
2
Train RMSE in = 5.736436
Valid RMSE in = 7.046701
Test RMSE in 2011= 3.651072
3
Train RMSE in = 5.763731
Valid RMSE in = 6.922666
Test RMSE in 2011= 3.593102
4
Train RMSE in = 5.74426
Valid RMSE in = 7.088115
Test RMSE in 2011= 3.596519
2012
0
Train RMSE in = 5.693477
Valid RMSE in = 6.972628
Test RMSE in 2012= 4.96152
1
Train RMSE in = 5.715264
Valid RMSE in = 6.991866
Test RMSE in 2012= 5.005154
2
Train RMSE in = 5.701855
Vali